In [ ]:
# Charger le modèle entraîné
cnn_model.load_state_dict(torch.load("/content/drive/MyDrive/fusion_model/quickdraw_cnn_10000.pth", map_location=torch.device('cpu')))
cnn_model.eval()

# Extraire les features d’une image
def extract_image_features(image_vector):
    image_tensor = torch.tensor(image_vector, dtype=torch.float32).view(1, 1, 28, 28).to(device)
    features = cnn_model(image_tensor).cpu().detach().numpy().reshape(128)
    return features

# Exemple d’extraction
image_features = extract_image_features(df.iloc[0]["image"])
print("Features extraites :", image_features, image_features.shape, df.iloc[0]["Catégorie"])  # (128,)

In [ ]:
# Charger Word2Vec français
w2v_model = Word2Vec.load("/content/drive/MyDrive/data/word2vec_fr.model")

In [ ]:
# Fonction pour encoder une question
def encode_question(question):
    words = simple_preprocess(question) #convert a document into a list of token.
    vectors = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(w2v_model.vector_size)

# Exemple
question = "Y a-t-il un avion ici ?"
question_vector = encode_question(question)
print("Feature de la question :", question_vector.shape)  # (200,)

In [ ]:
# Créer un DataLoader
batch_size = 32
test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

with torch.no_grad():
    mlp_model.eval()
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = mlp_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"✅ Précision sur le dataset de test : {accuracy}%")
    print("✅ Test terminé.")

In [ ]:
# Example usage:
def predict_answer(image_vector, question):
    # Extract image features
    image_features = extract_image_features(image_vector)

    # Encode the question
    question_features = encode_question(question)

    # Fuse features
    fused_features = fuse_features(image_features, question_features)

    # Convert to tensor
    input_tensor = torch.tensor(fused_features, dtype=torch.float32).unsqueeze(0).to(device)

    # Make prediction
    with torch.no_grad():
        output = mlp_model(input_tensor)
        _, predicted = torch.max(output.data, 1)

    # Return prediction (0 or 1)
    return predicted.item()

# Example prediction
image_index = 1 # Choose an image index
image_vector = df.iloc[image_index]["image"]
question = "Peux-tu me dire si c'est un airplane ?"

prediction = predict_answer(image_vector, question)
print(prediction)
print(f"Question: {question}")
print(f"Prediction: {'Oui' if prediction == 1 else 'Non'}")

# Display the image
pic = image_vector.reshape(28, 28)
plt.imshow(pic, cmap='gray')
plt.title(f"Actual Category: {df.iloc[image_index]['Catégorie']}")
plt.show()